Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Train Locally
In this notebook, you will perform the following using Azure Machine Learning.
* Load workspace.
* Configure & execute a local run in a user-managed Python environment.
* Configure & execute a local run in a system-managed Python environment.
* Configure & execute a local run in a Docker environment.
* Register model for operationalization.

In [1]:
import os
from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from dotenv import set_key, get_key, find_dotenv

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Initialize Workspace

Initialize a workspace object from persisted configuration file.

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

Found the config file in: /datadrive/MachineLearningNotebooks/mlaksdeployment/aml_config/config.json
fboyluamlsdkws
fboyluamlsdkrg
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


## Create An Experiment
**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [4]:
experiment_name = "mlaks-train-on-local"
exp = Experiment(workspace=ws, name=experiment_name)

## View `create_model.py`

The script that trains the model `create_model.py` is already created for you. Let's check its contents.

In [5]:
with open("./scripts/create_model.py", "r") as f:
    print(f.read())

import os
import argparse
import pandas as pd
import lightgbm as lgb
from sklearn.feature_extraction import text
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.externals import joblib
from ItemSelector import ItemSelector
from label_rank import label_rank
from azureml.core import Run


if __name__ == '__main__':
    # Define the arguments.
    parser = argparse.ArgumentParser(description='Fit and evaluate a model based on train-test datasets.')
    parser.add_argument('-d', '--train_data', help='the training dataset name', default='balanced_pairs_train.tsv')
    parser.add_argument('-t', '--test_data', help='the test dataset name', default='balanced_pairs_test.tsv')
    parser.add_argument('-i', '--estimators', help='the number of learner estimators', type=int, default=8000)
    parser.add_argument('--min_child_samples', help='the minimum number of samples in a child(leaf)', type=int, default=20)
    parser.add_argument('-v','--verbose', help='the verbo

Note `create_model.py` also references a `ItemSelector.py` and `label_rank.py` file. Let's check those scripts as well.

In [6]:
with open('./scripts/ItemSelector.py', 'r') as f:
    print(f.read())

# From: http://scikit-learn.org/0.18/auto_examples/hetero_feature_union.html
from sklearn.base import BaseEstimator, TransformerMixin


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at provided
    key(s).

    The data are expected to be stored in a 2D data structure, where
    the first index is over features and the second is over samples,
    i.e.

    >> len(data[keys]) == n_samples

    Please note that this is the opposite convention to scikit-learn
    feature matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[keys]).  Examples include: a dict of lists, 2D numpy array,
    Pandas DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sampl

In [7]:
with open('./scripts/label_rank.py', 'r') as f:
    print(f.read())

# Copyright (C) Microsoft Corporation.  All rights reserved.

import numpy as np
import pandas as pd


def score_rank(scores):
    return pd.Series(scores).rank(ascending=False)


def label_index(label, label_order):
    loc = np.where(label == label_order)[0]
    if loc.shape[0] == 0:
        return None
    return loc[0]


def label_rank(label, scores, label_order):
    loc = label_index(label, label_order)
    if loc is None:
        return len(scores) + 1
    return score_rank(scores)[loc]



## Configure & Run


### User-managed environment
Below, we use a user-managed run, which means you are responsible to ensure all the necessary packages that are available in the Python environment you choose to run the script. We will use the environment created for this tutorial which has Azure ML SDK and other dependencies installed.

In [8]:
# Editing a run configuration property on-fly.
run_config_user_managed = RunConfiguration()

run_config_user_managed.environment.python.user_managed_dependencies = True

# Choose the specific Python environment of this tutorial by pointing to the Python path
run_config_user_managed.environment.python.interpreter_path = (
    "/anaconda/envs/MLAKSDeployAML/bin/python"
)

#### Submit script to run in the user-managed environment
Note that the whole `scripts` folder is submitted for execution, including the `ItemSelector.py` and `label_rank.py` files. The model will be written to `outputs` directory which is a special directory such that all content in this directory is automatically uploaded to your workspace. 

In [9]:
scrpt = "create_model.py"
args = [
    "--inputs",
    os.path.abspath("./data_folder"),
    "--outputs",
    "outputs",
    "--estimators",
    "8000",
    "--match",
    "20",
]

In [10]:
src = ScriptRunConfig(
    source_directory="./scripts",
    script=scrpt,
    arguments=args,
    run_config=run_config_user_managed,
)
run = exp.submit(src)

#### Get run history details

In [11]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
mlaks-train-on-local,mlaks-train-on-local_1544111816_2f2b8256,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


Block to wait till run finishes.

In [12]:
%%time
run.wait_for_completion(show_output=True)

RunId: mlaks-train-on-local_1544111816_2f2b8256

Streaming azureml-logs/80_driver_log.txt

Reading /datadrive/MachineLearningNotebooks/mlaksdeployment/data_folder/balanced_pairs_train.tsv
train: 132,500 rows with 5.00% matches
Training...
outputs/model.pkl size: 22.42 MB
Reading /datadrive/MachineLearningNotebooks/mlaksdeployment/data_folder/balanced_pairs_test.tsv
test: 297,570 rows with 0.55% matches
Testing...
Accuracy @1 = 52.60%
Accuracy @2 = 67.71%
Accuracy @3 = 75.11%
Mean Rank 4.9829


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service

Execution Summary
RunId: mlaks-train-on-local_1544111816_2f2b8256

CPU times: user 6.96 s, sys: 242 ms, total: 7.2 s
Wall time: 6min 7s


{'runId': 'mlaks-train-on-local_1544111816_2f2b8256',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-12-06T15:56:57.24036Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '880a78f1-e040-4af0-bc16-307a5b6a9fca'},
 'runDefinition': {'Script': 'create_model.py',
  'Arguments': ['--inputs',
   '/datadrive/MachineLearningNotebooks/mlaksdeployment/data_folder',
   '--outputs',
   'outputs',
   '--estimators',
   '8000',
   '--match',
   '20'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': '/anaconda/envs/MLAKSDeployAML/bin/python',
    'UserManagedDependencies': True,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2', {'pip': ['azureml-defaults']}]},
    'CondaDependenciesFile':

Let's check that the model is now available in your workspace.

In [13]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'azureml-logs/80_driver_log.txt',
 'outputs/model.pkl',
 'outputs/labels.txt',
 'outputs/inst.txt',
 'driver_log',
 'azureml-logs/azureml.log']

Let's retrive the accuarcy of the model from run logs by querying the run metrics.

In [14]:
run.get_metrics()

{'Accuracy @1': 0.5259938837920489,
 'Accuracy @2': 0.6770642201834862,
 'Accuracy @3': 0.7510703363914373,
 'Mean Rank': 4.982874617737003}

### System-managed environment
You can also ask the system to build a new conda environment and execute your scripts in it. The environment is built once and will be reused in subsequent executions as long as the conda dependencies remain unchanged. 

In [15]:
run_config_system_managed = RunConfiguration()
run_config_system_managed.environment.python.user_managed_dependencies = False
run_config_system_managed.auto_prepare_environment = True

Let's specifiy the conda and pip dependencies.

In [16]:
# Specify conda dependencies with scikit-learn and pandas
conda_pack = ["scikit-learn==0.19.1", "pandas==0.23.3"]
requirements = ["lightgbm==2.1.2", "azureml-defaults"]

In [17]:
cd = CondaDependencies.create(conda_packages=conda_pack, pip_packages=requirements)
run_config_system_managed.environment.python.conda_dependencies = cd

#### Submit script to run in the system-managed environment
A new conda environment is built based on the conda dependencies object. If you are running this for the first time,  this might take up to 5 minutes. But this conda environment is reused so long as you don't change the conda dependencies.

In [18]:
src = ScriptRunConfig(
    source_directory="./scripts",
    script=scrpt,
    arguments=args,
    run_config=run_config_system_managed,
)
run = exp.submit(src)

#### Get run history details

In [19]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
mlaks-train-on-local,mlaks-train-on-local_1544113387_3850ce71,azureml.scriptrun,Preparing,Link to Azure Portal,Link to Documentation


Block and wait till run finishes.

In [20]:
%%time
run.wait_for_completion(show_output = True)

RunId: mlaks-train-on-local_1544113387_3850ce71

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Running ['conda', '--version']
Creating Conda environment...
Logging experiment preparation status in history service.
Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
  Using cached https://files.pythonhosted.org/packages/24/8d/783679697bdf1585dc96acfe7c8538d6fab33ad80e827cadb7b413e61da6/lightgbm-2.1.2-py2.py3-none-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f5/ce/3443d87ba4735de037003413063b7d5d77b00543b787c495b0ab682be254/azureml_core-1.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/c8/7848a0dd85158930b859eb8be1e38fc76a91f0a040d491723ebb356d7358/applicationinsights-0.11.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/d3/055ce7a

  Using cached https://files.pythonhosted.org/packages/c4/0c/c562be95a9a2ed52454f598571cf300b1114d0db2aa27f5b8ed3bb9cd0c0/azure_nspkg-3.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b3/c2/af4b47845f27dc7d206ed4908b9e580f8bc94a4b2f3956a0d87c40719d90/azure_mgmt_nspkg-3.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/9f/b36f7774ff5ea8e428fdcfc4bb332c39ee5b9362ddd3d40d9516a55221b2/isodate-0.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/94/e7/c250d122992e1561690d9c0f7856dadb79d61fd4bdd0e598087dce607f6c/requests_oauthlib-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/c9/cdeeeabc550848e2a07caa66cba28aa057d23b6feaa824ceafd32c3f2226/azure_cli_command_modules_nspkg-2.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6d/c0/47db8f624f3e4e2f3f27be03a93379d1ba16a1450a7b1aacfa0366e2c0dd/cffi-1.11.5-cp36-cp36m-manylinux1_x86_64.whl
  Using c

Streaming log file azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Reading /datadrive/MachineLearningNotebooks/mlaksdeployment/data_folder/balanced_pairs_train.tsv
train: 132,500 rows with 5.00% matches
Training...
outputs/model.pkl size: 22.42 MB
Reading /datadrive/MachineLearningNotebooks/mlaksdeployment/data_folder/balanced_pairs_test.tsv
test: 297,570 rows with 0.55% matches
Testing...
Accuracy @1 = 52.60%
Accuracy @2 = 67.71%
Accuracy @3 = 75.11%
Mean Rank 4.9823


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service

Execution Summary
RunId: mlaks-train-on-local_1544113387_3850ce71

CPU times: user 7.68 s, sys: 252 ms, total: 7.93 s
Wall time: 6min 35s


{'runId': 'mlaks-train-on-local_1544113387_3850ce71',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-12-06T16:23:43.262309Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '880a78f1-e040-4af0-bc16-307a5b6a9fca'},
 'runDefinition': {'Script': 'create_model.py',
  'Arguments': ['--inputs',
   '/datadrive/MachineLearningNotebooks/mlaksdeployment/data_folder',
   '--outputs',
   'outputs',
   '--estimators',
   '8000',
   '--match',
   '20'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
      {'pip': ['lightgbm==2.1.2', 'azureml-defaults==1.0.2']},
      'scikit-learn==0.19.1',


In [21]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'azureml-logs/80_driver_log.txt',
 'outputs/labels.txt',
 'outputs/model.pkl',
 'outputs/inst.txt',
 'driver_log',
 'azureml-logs/azureml.log']

In [22]:
run.get_metrics()

{'Accuracy @1': 0.5259938837920489,
 'Accuracy @2': 0.6770642201834862,
 'Accuracy @3': 0.7510703363914373,
 'Mean Rank': 4.982262996941896}

### Docker-based execution
**IMPORTANT**: You must have Docker engine installed locally in order to use this execution mode. If your kernel is already running in a Docker container, such as **Azure Notebooks**, this mode will **NOT** work.

You can also ask the system to pull down a Docker image and execute your scripts in it. We will use the `continuumio/miniconda3` image for that purpose.

In [23]:
run_config_docker = RunConfiguration()
run_config_docker.environment.python.user_managed_dependencies = False
run_config_docker.auto_prepare_environment = True
run_config_docker.environment.docker.enabled = True
run_config_docker.environment.docker.base_image = "continuumio/miniconda3"

# Specify conda and pip dependencies
cd = CondaDependencies.create(conda_packages=conda_pack, pip_packages=requirements)
run_config_docker.environment.python.conda_dependencies = cd

Here, we map the local `data_folder` that includes the training and testing data to the docker container using `-v` flag.

In [24]:
host_dir = os.path.abspath("./data_folder")
container_dir = "/data_folder"
docker_arg = "{}:{}".format(host_dir, container_dir)

This time the run will use the mapped `data_folder` inside the docker container to find the data files.

In [25]:
args = [
    "--inputs",
    "/data_folder",
    "--outputs",
    "outputs",
    "--estimators",
    "8000",
    "--match",
    "20",
]

In [26]:
run_config_docker.environment.docker.arguments.append("-v")
run_config_docker.environment.docker.arguments.append(docker_arg)

In [27]:
src = ScriptRunConfig(
    source_directory="./scripts",
    script=scrpt,
    arguments=args,
    run_config=run_config_docker,
)

In [28]:
 run = exp.submit(src)

In [29]:
#Get run history details
run

Experiment,Id,Type,Status,Details Page,Docs Page
mlaks-train-on-local,mlaks-train-on-local_1544113913_cc40f91f,azureml.scriptrun,Preparing,Link to Azure Portal,Link to Documentation


In [30]:
%%time
run.wait_for_completion(show_output=True)

RunId: mlaks-train-on-local_1544113913_cc40f91f

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Logging experiment preparation status in history service.
Running: ['sudo', 'docker', 'build', '-f', 'azureml-setup/Dockerfile', '-t', 'azureml/azureml_d3c93f231fcf9c4f309db7e1136bfddc', '.']
Sending build context to Docker daemon  250.9kB
Step 1/13 : FROM continuumio/miniconda3
 ---> d3c252f8727b
Step 2/13 : USER root
 ---> Using cache
 ---> e6e67f68b353
Step 3/13 : RUN mkdir -p $HOME/.cache
 ---> Using cache
 ---> f3e563489a71
Step 4/13 : WORKDIR /
 ---> Using cache
 ---> 39e962b6c337
Step 5/13 : COPY azureml-setup/99brokenproxy /etc/apt/apt.conf.d/
 ---> Using cache
 ---> f1a59c150225
Step 6/13 : RUN if dpkg --compare-versions `conda --version | grep -oE '[^ ]+$'` lt 4.4.0; then conda install conda==4.4.11; fi
 ---> Using cache
 ---> de364fdae1be
Step 7/13 : COPY azureml-setup/mutated_conda_dependencies.yml azureml-setup/mutated_conda_depende

  Running setup.py bdist_wheel for SecretStorage: started
  Running setup.py bdist_wheel for SecretStorage: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/4e/5b/1b/be8c8a830a0243af85b2946a0aece2c6743d7f7f946977ed67
  Running setup.py bdist_wheel for pathspec: started
  Running setup.py bdist_wheel for pathspec: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Running setup.py bdist_wheel for pyyaml: started
  Running setup.py bdist_wheel for pyyaml: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
  Running setup.py bdist_wheel for tabulate: started
  Running setup.py bdist_wheel for tabulate: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
  Running setup.py bdist_wheel for antlr4-python3-runtime: sta

Removing intermediate container 2d1af880dfc7
 ---> 4e7297391335
Step 13/13 : CMD ["bash"]
 ---> Running in 7262702f417e
Removing intermediate container 7262702f417e
 ---> 4e22cdb9a748
Successfully built 4e22cdb9a748
Successfully tagged azureml/azureml_d3c93f231fcf9c4f309db7e1136bfddc:latest
Logging experiment running status in history service.
Running: ['sudo', 'docker', 'run', '--name', 'mlaks-train-on-local_1544113913_cc40f91f', '--rm', '-v', '/tmp/azureml_runs/mlaks-train-on-local_1544113913_cc40f91f:/azureml-run', '-e', 'AZUREML_DOCKERFILE_PATH=azureml-setup/Dockerfile', '-e', 'AZUREML_CONDA_ENVIRONMENT_EXISTENCE_INDICATOR=$HOME/.azureml/locks/azureml_6f4c7ec6b9b63f85336804cf7368299c', '-e', 'AZUREML_CONDA_ENVIRONMENT_PATH=/azureml-envs/azureml_6f4c7ec6b9b63f85336804cf7368299c', '-e', 'AZUREML_CONDA_LOCK_PATH=$HOME/.azureml/locks/azureml_conda_lock', '-e', 'AZUREML_USER_CONDA_DIRECTORY=$HOME/.azureml', '-e', 'EXAMPLE_ENV_VAR=EXAMPLE_VALUE', '-e', 'AZUREML_CONTEXT_MANAGER_OUTPUTCOLL

{'runId': 'mlaks-train-on-local_1544113913_cc40f91f',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-12-06T16:34:46.723701Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '880a78f1-e040-4af0-bc16-307a5b6a9fca'},
 'runDefinition': {'Script': 'create_model.py',
  'Arguments': ['--inputs',
   '/data_folder',
   '--outputs',
   'outputs',
   '--estimators',
   '8000',
   '--match',
   '20'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
      {'pip': ['lightgbm==2.1.2', 'azureml-defaults==1.0.2']},
      'scikit-learn==0.19.1',
      'pandas==0.23.3']},
    'CondaDependenciesFil

In [31]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'azureml-logs/80_driver_log.txt',
 'outputs/model.pkl',
 'outputs/labels.txt',
 'outputs/inst.txt',
 'driver_log',
 'azureml-logs/azureml.log']

In [32]:
run.get_metrics()

{'Accuracy @1': 0.5259938837920489,
 'Accuracy @2': 0.6770642201834862,
 'Accuracy @3': 0.7510703363914373,
 'Mean Rank': 4.982262996941896}

## Register Model

We now register the model with the workspace so that we can later deploy the model.

In [40]:
# supply a model name, and the full path to the serialized model file.
model = run.register_model(
    model_name="question_match_model", model_path="./outputs/model.pkl"
)

Deprecated, use RunHistoryFacade.assets instead.


In [41]:
print(model.name, model.version, model.url, sep="\n")

question_match_model
3
aml://asset/f30875e852114ff58e2664301016b274


In [42]:
model_version = str(model.version)
set_key(env_path, "model_version", model_version)

(True, 'model_version', '3')

Next, we [develop the scoring script](03_Develop_Scoring_Script.ipynb) for this model.